# Derive Vertex structure from Trajectories.

For each frame in the trajectories obtained with [ParticleTracking](ParticleTracking.ipynb), we will convert this to a Colloidal Ice, and then get the corresponding vertices. This is based on the processing of [ReanalyseFirstFrames](ReanalyseFirstFrames.ipynb). 

In [1]:
import os
import sys
import glob
sys.path.insert(0, './')

import icenumerics as ice
from icenumerics.geometry import transformations as tr

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm_notebook
import support as sp

idx = pd.IndexSlice
ureg = ice.ureg

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

In [3]:
directory = "C:/Users/aortiza/Desktop/ShearedSquare/Theta45"

All trajectories are stored in a single dataset. 

In [4]:
%%time 
trj = pd.read_csv(os.path.join(directory,"trajectories.dat"), sep = "\t", index_col = [0,1,2])

C:\Users\aortiza\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 31.9 s


In [5]:
names = trj.index.get_level_values("filename").unique()

I will use the simulation framework of [icenumerics](https://aortiza.github.io/icenumerics/) to create a colloidal ice structure that can be superimposed on the particles' locations. 

In [6]:
# These are the base objects that are used to construct a colloidal ice structure over the image of the particles.
theta = 45
alpha = (90-theta)/180*np.pi
nx = 7
ny = 11

particle = ice.particle()

trap = ice.trap(trap_sep = 10*ureg.um,
               height = 80*ureg.pN*ureg.nm,
               stiffness = 6e-4*ureg.pN/ureg.nm)

square_spins = ice.spins()
v = ice.vertices()

We will use the parameters obtained in [AlignImagesAndLattices](AlignImagesAndLattices.ipynb). These define a transformation to match the positions of the colloids and those of the colloids_in_traps

In [7]:
parameters = pd.read_csv(os.path.join(directory,"parameters.dat"),sep = "\t", index_col = [0])

The following functions make the transformation of the colloidal ice, and they match a particle to a colloid_in_trap, by minimizing the distance betwen them. The matching function is defined in the [support](support.py) file. 

In [8]:
def particle_locations_to_colloids(locations,filename):
    """ converts locations into a colloidal ice """
    entry = parameters.loc[filename]

    square_spins.create_lattice("square",[nx,ny],lattice_constant = 30*ureg.um)
    sheared_spins = tr.shear(square_spins,alpha)

    sheared_spins = tr.rotate(sheared_spins,(entry[["angle"]].values[0])*ureg.deg)
    sheared_spins = tr.scale(sheared_spins,entry[["scale"]].values[0]*np.array([1,1]))
    sheared_spins = tr.translate(sheared_spins,entry[["x_offset","y_offset"]].values*ureg.um)
    
    col = ice.colloidal_ice(sheared_spins, particle, trap, height_spread = 0, susceptibility_spread = 0.1)
    
    sp.match_colloid_frame(col,locations)

    return col

This function gets rid of vertices which are not inside a box. 

In [9]:
def filter_vertices(vert_frame,box):
    """ Filters out vertices located outside of a box"""
    theta = box.angle/180*np.pi
    
    X = vert_frame[:].LocationX
    Y = vert_frame[:].LocationY

    Xt = X*np.cos(theta)-Y*np.sin(theta)
    Yt = X*np.sin(theta)+Y*np.cos(theta)
    
    x0,y0 = box.xy
    
    xf = x0 + box.get_width()
    yf = y0 + box.get_height()

    crit = (Xt>x0) & (Xt<xf) & (Yt>y0) & (Yt<yf)
    return vert_frame[crit]

The colloids to vertices function is defined within icenumerics. The following function just puts everything together.

In [10]:
left = 30
right = 30
bottom = 60
top = 60

video_shape = [512,640]
def particle_locations_to_vertices(locations):
    """ converts particle locations into an array of vertices """
    
    filename = locations.index.get_level_values("filename").unique()[0]
    
    col = particle_locations_to_colloids(locations,filename)

    v = ice.vertices()
    
    v.colloids_to_vertices(col)
    
    vertices = sp.vertices_array_to_pd(v)
    
    entry = parameters.loc[filename]

    box = mpl.patches.Rectangle(
        (left+(entry.x_offset-620),bottom+(entry.y_offset+10)),
        (video_shape[1]-left-right),(video_shape[0]-bottom-top),
        angle = entry.angle-90, alpha = 0.2)

    vert_frame = filter_vertices(vertices, box)

    return vert_frame

In [11]:
from tqdm import tqdm
tqdm.pandas()

We now apply this to every file. We skip 9/10 frames because we don't really need that many, and it is a slow process.

In [12]:
vertices = trj.loc[idx[names,::10,:],:].groupby(["filename","frame"]).progress_apply(particle_locations_to_vertices)

  0%|                                      | 1/13730 [00:01<3:55:18,  1.03s/it]C:\Users\aortiza\Anaconda3\lib\site-packages\pint\quantity.py:1377: UnitStrippedWarning: The unit of the quantity is stripped.
  warnings.warn("The unit of the quantity is stripped.", UnitStrippedWarning)
100%|██████████████████████████████████| 13730/13730 [2:50:31<00:00,  1.36it/s]


## Add field data to vertex dataset

From the field data of the trajectories, we obtain the field data of the vertices. 

In [13]:
field = trj.filter(["field"]).groupby(["filename","frame"]).mean()

In [14]:
file_index = vertices.index.get_level_values("filename")
frame_index = vertices.index.get_level_values("frame")

In [15]:
vertices["field"] = field.loc[idx[file_index,frame_index],"field"]

In [16]:
vertices.head()

Coordination  Charge  \
filename                              frame id                         
E5_M8_teta45_10um_2019_06_04_15_32_20 0     15             4       2   
                                            17             4       2   
                                            18             4       0   
                                            19             4      -2   
                                            21             4       0   

                                                 DipoleX   DipoleY  \
filename                              frame id                       
E5_M8_teta45_10um_2019_06_04_15_32_20 0     15  1.394330 -1.433821   
                                            17 -1.394330  1.433821   
                                            18  1.422255  0.565984   
                                            19  0.027924  1.999805   
                                            21  1.422255  0.565984   

                                                 LocationX   LocationY  field  
filename                              frame id                                 
E5_M8_teta45_10um_2019_06_04_15_32_20 0     15  291.459582   92.536974    0.0  
                                            17  461.130516  392.734273    0.0  
                                            18  235.991654   70.463604    0.0  
                                            19  405.662588  370.660903    0.0  
                                            21  572.066372  436.881012    0.0

Now we save the vertices to a .dat. The next steps are done in the notebook [VertexCount](VertexCount.ipynb).

In [17]:
vertices.to_csv(os.path.join(directory,"vertices_new.dat"),sep = "\t")